# LESSON 1: Uninformed Search Strategies

In this first session we will work on uninformed search. 

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

In order to use the environment we need first to import the packages of OpenAI Gym. Notice that due to the structure of this repository, we need to add the parent directory to the path

In [4]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs

### Assignment 1: Breadth-First Search (BFS)

Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, you are required to implement both tree_search and graph_search versions of BFS that will be called by the generic bfs. 

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory at the same time.

After the correctness of your implementations have been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_TreeSearch(problem)
- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here the pseudo-code form the book **Artificial Intelligence: A Modern Approach** for the *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here the pseudo-code form the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/bfs.png" width="600">

The next two functions have to be implemented

In [18]:
def BFS_TreeSearch(problem):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 0
    space_cost = 1
    frontier = NodeQueue()
    frontier.add(node)
    while True:
        if frontier.is_empty():
            return failure
        node = frontier.remove()
        time_cost += 1
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state, action), node)
            space_cost +=1
            if child.state == problem.goalstate:
                return build_path(child), time_cost, space_cost
            frontier.add(child)

In [19]:
def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 0
    space_cost = 1
    
    frontier = NodeQueue()
    explored = NodeQueue()
    frontier.add(node)
    while True:
        if frontier.is_empty():
            return failure
        node = frontier.remove()
        time_cost += 1
        explored.add(node)
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state, action), node)
            space_cost +=1
            if child.state not in explored or child.state not in frontier:
                if child.state == problem.goalstate:
                    return build_path(child), time_cost, space_cost
                frontier.add(child) 

The following code calls your tree search and graph search version of BFS and prints the results

In [20]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

print("\n----------------------------------------------------------------")
print("\tBFS TREE SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_ts, environment)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}".format(memory_ts))

print("\n----------------------------------------------------------------")
print("\tBFS GRAPH SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_gs, environment)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}".format(memory_gs))


----------------------------------------------------------------
	BFS TREE SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 25931
Max n° of nodes in memory: 103723

----------------------------------------------------------------
	BFS GRAPH SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 171
Max n° of nodes in memory: 683


Correct results can be found [here](lesson_1_results.txt).

### Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, you are required to implement *DLS* in the graph search version, *DLS* in the tree search version and the final *Iterative_DLS*.

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations have been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- Recursive_DLS_TreeSearch(node, problem, limit)
- Recursive_DLS_GraphSearch(node, problem, limit, explored)
- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here the pseudo-code form the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search* (Tree Search Version) and *Iterative deepening depth-first search (Tree Search Version)*:
<img src="images/dls.png" width="600">
<img src="images/ids.png" width="600">

Note that **Node** has a useful variable that can be set in the constructor and can be used to track the depth of a node in the path (and consequently of the recursion stack of IDS): pathcost. If the root node has a pathcost of 0, its children will have a pathcost increased by 1.

In [ ]:
start = environment.startstate
root = Node(start)  # parent = None and pathcost = 0 as default
child = Node(environment.sample(start, 0), root, root.pathcost + 1)  # pathcost is the third argument
print("Root pathcost: {}\tChild pathcost: {}".format(root.pathcost, child.pathcost))

In [ ]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    node = Node(problem.startstate, None)
    return RDLS_Function(node, problem, limit, set())

The next three functions have to be implemented:

In [ ]:
def Recursive_DLS_GraphSearch(node, problem, limit, explored):
    """
    Recursive DLS
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        explored: completely explored nodes
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    #
    # YOUR CODE HERE ...
    #
    explored.add(node.state)
    space_cost = node.pathcost
    time_cost = 1 
    #
    # YOUR CODE HERE ...
    #
    return result[0], time_cost, space_cost
    return "cut_off", time_cost, space_cost
    return "failure", time_cost, space_cost

In [ ]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None):
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    #
    # YOUR CODE HERE ...
    #
    space_cost = node.pathcost
    time_cost = 1 
    #
    # YOUR CODE HERE ...
    #
    return result[0], time_cost, space_cost
    return "cut_off", time_cost, space_cost
    return "failure", time_cost, space_cost

In [ ]:
def IDS(problem, DLS_Function):
    """
    Iteartive_DLS DLS

    Args:
    problem: OpenAI Gym environment

    Returns:
    (path, time_cost, space_cost): solution as a path and stats.
    """

    total_time_cost = 0
    total_space_cost = 1
    for i in zero_to_infinity():
        #
        # YOUR CODE HERE ...
        #
        return 0, 0, 0, 0 # Placeholder
    return solution_dls, total_time_cost, total_space_cost, i

The following code calls your version of IDS and prints the results:

In [ ]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)
solution_gs, time_gs, memory_gs, iterations_gs = IDS(environment, Recursive_DLS_GraphSearch)

print("\n----------------------------------------------------------------")
print("\tIDS TREE SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Necessary Iterations: {}".format(iterations_ts))
print("Solution: {}".format(solution_2_string(solution_ts, environment)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}".format(memory_ts))
        
print("\n----------------------------------------------------------------")
print("\tIDS GRAPH SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Necessary Iterations: {}".format(iterations_gs))
print("Solution: {}".format(solution_2_string(solution_gs, environment)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}".format(memory_gs))

Correct results can be found [here](lesson_1_results.txt).

### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?